线上效果0.4719，21名，受邀请分享一下思路~

总体思路：分别使用LightGBM，xgboost，gbdt，catboost建立多个个体学习器（加入bagging的策略，对数据随机采样），对最终学习器的输出使用岭回归进一步提升精度。代码如下。

改进点：
1.可以在详细分析一下字段，可以考虑对字段进行特殊处理。
2.超参数还可以调，我没有使用网格搜索，只是简单的进行的调参。
3.如果单纯为了提高精度，可以更高随机种子，多试几次


In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("happiness_train_complete.csv",encoding="GB2312")
df = df.sample(frac=1,replace=False,random_state=11)
df.reset_index(inplace=True)
df = df[df["happiness"]>0]
Y = df["happiness"]
df["survey_month"] = df["survey_time"].map(lambda line:line.split(" ")[0].split("/")[1]).astype("int64")
df["survey_day"] = df["survey_time"].map(lambda line:line.split(" ")[0].split("/")[2]).astype("int64")
df["survey_hour"] = df["survey_time"].map(lambda line:line.split(" ")[1].split(":")[0]).astype("int64")
X = df.drop(columns=["id","index","happiness","survey_time","edu_other","property_other","invest_other"])

In [16]:
from sklearn.model_selection import train_test_split
from lightgbm.sklearn import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")

In [17]:
kfold = KFold(n_splits=15, shuffle = True, random_state= 12)

In [18]:
model = LGBMRegressor(n_jobs=-1,learning_rate=0.051,
                      n_estimators=400,
                      num_leaves=11,
                      reg_alpha=2.0, 
                      reg_lambda=2.1,
                      min_child_samples=6,
                      min_split_gain=0.5,
                      colsample_bytree=0.2
                     )

In [19]:
mse = []
i=0
for train, test in kfold.split(X):#第一个是训练集的索引，第二个是验证集的索引。
    X_train = X.iloc[train]
    y_train = Y.iloc[train]
    X_test = X.iloc[test]
    y_test = Y.iloc[test]
    model.fit(X_train,y_train)
#     model2.fit(model.predict(X_train,pred_leaf=True),y_train)
#     y_pred = model2.predict(model.predict(X=X_test,pred_leaf=True))
    y_pred = model.predict(X=X_test)
    e = mean_squared_error(y_true=y_test,y_pred=y_pred)
    mse.append(e)
    print(e)
    joblib.dump(filename="light"+str(i),value=model)
    i+=1
print("lightgbm",np.mean(mse),mse)

0.5246388446214709
0.44853289149980635
0.39798888520938375
0.4653840765647241
0.4198606821280977
0.4122033421642622
0.502844325280201
0.4939943554900641
0.45983459406105315
0.49773527223459996
0.44428059718587704
0.4819013944449417
0.4556318435524804
0.46869886997155735
0.4244474502100046
lightgbm 0.4598651616412349 [0.5246388446214709, 0.44853289149980635, 0.39798888520938375, 0.4653840765647241, 0.4198606821280977, 0.4122033421642622, 0.502844325280201, 0.4939943554900641, 0.45983459406105315, 0.49773527223459996, 0.44428059718587704, 0.4819013944449417, 0.4556318435524804, 0.46869886997155735, 0.4244474502100046]


In [20]:
#CatBoostRegressor
import pandas as pd
import numpy as np
df = pd.read_csv("happiness_train_complete.csv",encoding="GB2312")
df = df.sample(frac=1,replace=False,random_state=11)
df.reset_index(inplace=True)

df = df[df["happiness"]>0]
Y = df["happiness"]
df["survey_month"] = df["survey_time"].map(lambda line:line.split(" ")[0].split("/")[1]).astype("int64")
df["survey_day"] = df["survey_time"].map(lambda line:line.split(" ")[0].split("/")[2]).astype("int64")
df["survey_hour"] = df["survey_time"].map(lambda line:line.split(" ")[1].split(":")[0]).astype("int64")
X = df.drop(columns=["id","index","happiness","survey_time","edu_other","property_other","invest_other"])


from sklearn.model_selection import train_test_split
from catboost import Pool, CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.externals import joblib
kfold = KFold(n_splits=15, shuffle = True, random_state= 12)
model = CatBoostRegressor(colsample_bylevel=0.1,thread_count=6,silent=True,iterations=800, 
                          depth=5, 
                          learning_rate=0.051, 
                          loss_function='RMSE',
                          l2_leaf_reg = 3)
mse = []
i=0
for train, test in kfold.split(X):
    X_train = X.iloc[train]
    y_train = Y.iloc[train]
    X_test = X.iloc[test]
    y_test = Y.iloc[test]
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    err = mean_squared_error(y_true=y_test,y_pred=y_pred)
    mse.append(err)
    print(err)
    joblib.dump(filename="cat"+str(i),value=model)
    i+=1
print("catboost",np.mean(mse),mse)

0.5140745902446223
0.44963717420374527
0.40583414150925673
0.46129771045939694
0.4155310816111115
0.40941501701181154
0.4998116621189004
0.48265208770060597
0.45761449468465004
0.4895659400203275
0.4398279303183797
0.48314758270880803
0.4551596579865996
0.46339945766808416
0.4220552064306169
catboost 0.4566015823117944 [0.5140745902446223, 0.44963717420374527, 0.40583414150925673, 0.46129771045939694, 0.4155310816111115, 0.40941501701181154, 0.4998116621189004, 0.48265208770060597, 0.45761449468465004, 0.4895659400203275, 0.4398279303183797, 0.48314758270880803, 0.4551596579865996, 0.46339945766808416, 0.4220552064306169]


In [21]:
#xgboost
import pandas as pd
import numpy as np
df = pd.read_csv("happiness_train_complete.csv",encoding="GB2312")
df = df.sample(frac=1,replace=False,random_state=11)
df.reset_index(inplace=True)
df = df[df["happiness"]>0]
Y = df["happiness"]
df["survey_month"] = df["survey_time"].map(lambda line:line.split(" ")[0].split("/")[1]).astype("int64")
df["survey_day"] = df["survey_time"].map(lambda line:line.split(" ")[0].split("/")[2]).astype("int64")
df["survey_hour"] = df["survey_time"].map(lambda line:line.split(" ")[1].split(":")[0]).astype("int64")
X = df.drop(columns=["id","index","happiness","survey_time","edu_other","property_other","invest_other"])

from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
from sklearn.model_selection import KFold
kfold = KFold(n_splits=15, shuffle = True, random_state= 11)
model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.1,
       colsample_bytree=0.971, gamma=0.11, learning_rate=0.069, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=499,
       n_jobs=-1, nthread=50, objective='reg:linear', random_state=0,
       reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1.0)

mse = []
i = 0
for train, test in kfold.split(X):
    X_train = X.iloc[train]
    y_train = Y.iloc[train]
    X_test = X.iloc[test]
    y_test = Y.iloc[test]

    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    xg_mse = mean_squared_error(y_true=y_test,y_pred=y_pred)
    mse.append(xg_mse)
    print("xgboost",xg_mse)
    joblib.dump(filename="xg"+str(i),value=model)
    i+=1
print("xgboost",np.mean(mse),mse)


xgboost 0.45450514377237855
xgboost 0.4704459080976553
xgboost 0.45398107906457325
xgboost 0.4752842442418985
xgboost 0.44166640422869086
xgboost 0.4740424619970855
xgboost 0.414644665043313
xgboost 0.451792770308976
xgboost 0.47684027202123286
xgboost 0.49329761555264395
xgboost 0.4755900949442348
xgboost 0.4366128914483463
xgboost 0.5073443590751492
xgboost 0.42703606159588203
xgboost 0.4782073499962756
xgboost 0.46208608809255575 [0.45450514377237855, 0.4704459080976553, 0.45398107906457325, 0.4752842442418985, 0.44166640422869086, 0.4740424619970855, 0.414644665043313, 0.451792770308976, 0.47684027202123286, 0.49329761555264395, 0.4755900949442348, 0.4366128914483463, 0.5073443590751492, 0.42703606159588203, 0.4782073499962756]


In [22]:
#gbdt
import pandas as pd
import numpy as np
df = pd.read_csv("happiness_train_complete.csv",encoding="GB2312")
df = df.sample(frac=1,replace=False,random_state=11)
df.reset_index(inplace=True)
df = df[df["happiness"]>0]
Y = df["happiness"]
df["survey_month"] = df["survey_time"].map(lambda line:line.split(" ")[0].split("/")[1]).astype("int64")
df["survey_day"] = df["survey_time"].map(lambda line:line.split(" ")[0].split("/")[2]).astype("int64")
df["survey_hour"] = df["survey_time"].map(lambda line:line.split(" ")[1].split(":")[0]).astype("int64")
X = df.drop(columns=["id","index","happiness","survey_time","edu_other","property_other","invest_other"])

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
from sklearn.model_selection import KFold
kfold = KFold(n_splits=15, shuffle = True, random_state= 12)
model = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.051, loss='ls', max_depth=4, max_features=10,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=600, presort='auto', random_state=3,
             subsample=0.98, verbose=0, warm_start=False)

X.fillna(-8,inplace=True)

mse = []
i = 0
for train, test in kfold.split(X):
    X_train = X.iloc[train]
    y_train = Y.iloc[train]
    X_test = X.iloc[test]
    y_test = Y.iloc[test]

    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    gbdt_mse = mean_squared_error(y_true=y_test,y_pred=y_pred)
    mse.append(gbdt_mse)
    print("gbdt",gbdt_mse)
    joblib.dump(filename="gbdt"+str(i),value=model)
    i+=1
print("gbdt",np.mean(mse),mse)


gbdt 0.5189500888507569
gbdt 0.4570418891557324
gbdt 0.4033749775774016
gbdt 0.45943222208389095
gbdt 0.4286129037296931
gbdt 0.42570638271447414
gbdt 0.5043808541694681
gbdt 0.5032374595207704
gbdt 0.45540367596333403
gbdt 0.49633563078272347
gbdt 0.45106702861984144
gbdt 0.4972331370414904
gbdt 0.4568532289923147
gbdt 0.46964687347159
gbdt 0.43481614328190643
gbdt 0.4641394997303593 [0.5189500888507569, 0.4570418891557324, 0.4033749775774016, 0.45943222208389095, 0.4286129037296931, 0.42570638271447414, 0.5043808541694681, 0.5032374595207704, 0.45540367596333403, 0.49633563078272347, 0.45106702861984144, 0.4972331370414904, 0.4568532289923147, 0.46964687347159, 0.43481614328190643]


In [25]:
#带权平均融合CatBoostRegressor + xgboost + gbdt现有模型
import pandas as pd
import numpy as np
df = pd.read_csv("happiness_train_complete.csv",encoding="GB2312")
df = df.sample(frac=1,replace=False,random_state=2000)
df.reset_index(inplace=True)

df = df[df["happiness"]>0]
Y = df["happiness"]
df["survey_month"] = df["survey_time"].map(lambda line:line.split(" ")[0].split("/")[1]).astype("int64")
df["survey_day"] = df["survey_time"].map(lambda line:line.split(" ")[0].split("/")[2]).astype("int64")
df["survey_hour"] = df["survey_time"].map(lambda line:line.split(" ")[1].split(":")[0]).astype("int64")
X = df.drop(columns=["id","index","happiness","survey_time","edu_other","property_other","invest_other"])

from catboost import Pool, CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.externals import joblib
kfold = KFold(n_splits=10, shuffle = True, random_state= 110)
catmse = []
lightmse = []
xgmse = []
gbdtmse = []
lrmse = []
i = 0
for train, test in kfold.split(X):
    X_train = X.iloc[train]
    y_train = Y.iloc[train]
    X_test = X.iloc[test]
    y_test = Y.iloc[test]
    
    cat = joblib.load(filename="cat"+str(i))
    light = joblib.load(filename="light"+str(i))
    xg = joblib.load(filename="xg"+str(i))
    gbdt = joblib.load(filename="gbdt"+str(i))
    
    catX = cat.predict(X_test)
    cat_mse = mean_squared_error(y_true=y_test,y_pred=catX)
    print("\ncat mse:",cat_mse)
    catmse.append(cat_mse)
    
#     X_test2 = X_test.drop(columns=["survey_day"])
#     lightX = light.predict(X_test2)
#     light_mse = mean_squared_error(y_true=y_test,y_pred=lightX)
#     print("light mse:",light_mse)
#     lightmse.append(light_mse)
    
    xgX = xg.predict(X_test)
    xg_mse = mean_squared_error(y_true=y_test,y_pred=xgX)
    print("xg mse:",xg_mse)
    xgmse.append(xg_mse)
    
    X_test2 = X_test.fillna(-8)
    gbdtX = gbdt.predict(X_test2)
    gbdt_mse = mean_squared_error(y_true=y_test,y_pred=gbdtX)
    print("gbdt mse:",gbdt_mse)
    gbdtmse.append(gbdt_mse)
    
    res = np.c_[catX,xgX,gbdtX]
    e = np.array([1/cat_mse,1/xg_mse,1/gbdt_mse])
    y_pred = np.sum(res*e,axis=1)/sum(e)
    lr_mse = mean_squared_error(y_true=y_test,y_pred=y_pred)
    print("lr mse:",lr_mse)
    lrmse.append(lr_mse)
    
    i+=1
    
print("\n\ncatmse:",np.mean(catmse))
# print("lightmse:",np.mean(lightmse))
print("xgmse:",np.mean(xgmse))
print("gbdtmse:",np.mean(gbdtmse))
print("lrmse:",np.mean(lrmse))

cat = CatBoostRegressor(colsample_bylevel=0.1,thread_count=6,silent=True,iterations=800, 
                          depth=5, 
                          learning_rate=0.051, 
                          loss_function='RMSE',
                          l2_leaf_reg = 3)
xg = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.1,
       colsample_bytree=0.971, gamma=0.11, learning_rate=0.069, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=499,
       n_jobs=-1, nthread=50, objective='reg:linear', random_state=0,
       reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1.0)
gbdt = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.051, loss='ls', max_depth=4, max_features=10,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=600, presort='auto', random_state=3,
             subsample=0.98, verbose=0, warm_start=False)
cat.fit(X,Y)
xg.fit(X,Y)
gbdt.fit(X.fillna(-8),Y)
    
df2 = pd.read_csv("happiness_test_complete.csv",encoding="GB2312")
df2["survey_month"] = df2["survey_time"].map(lambda line:line.split(" ")[0].split("/")[1]).astype("int64")
df2["survey_day"] = df2["survey_time"].map(lambda line:line.split(" ")[0].split("/")[2]).astype("int64")
df2["survey_hour"] = df2["survey_time"].map(lambda line:line.split(" ")[1].split(":")[0]).astype("int64")

out = df2[["id"]]
X = df2.drop(columns=["id","survey_time","edu_other","property_other","invest_other"])
X2 = X.drop(columns=["survey_day"])
catX = cat.predict(X)
xgX = xg.predict(X)
gbdtX = gbdt.predict(X.fillna(-8))

res = np.c_[catX,xgX,gbdtX]
e = np.array([1/np.mean(catmse),1/np.mean(xgmse),1/np.mean(gbdtmse)])
y_pred = np.sum(res*e,axis=1)/sum(e)
out["happiness"] = y_pred
out.to_csv("happiness_submit.csv",index=False)
print("done")
print(e)


cat mse: 0.3068473087722804
xg mse: 0.3594664784459855
gbdt mse: 0.284963304317945
lr mse: 0.3099544960184681

cat mse: 0.26379334475643545
xg mse: 0.3090892975232685
gbdt mse: 0.2428776859214876
lr mse: 0.26508254457664676

cat mse: 0.3122819689309576
xg mse: 0.36448487487557585
gbdt mse: 0.2899090649776326
lr mse: 0.31457046922213133

cat mse: 0.31720119359048826
xg mse: 0.37212291504245154
gbdt mse: 0.2859330265818493
lr mse: 0.3166726800712165

cat mse: 0.2981851002570271
xg mse: 0.35301986484474096
gbdt mse: 0.2716722895520111
lr mse: 0.2996516041696275

cat mse: 0.27755281034439633
xg mse: 0.31740095174973676
gbdt mse: 0.2581780452897828
lr mse: 0.2787925340892307

cat mse: 0.32922390966018006
xg mse: 0.37127795635373706
gbdt mse: 0.29988440926202026
lr mse: 0.3259930877693204

cat mse: 0.29257941335360815
xg mse: 0.3289029414071347
gbdt mse: 0.26512869295338437
lr mse: 0.28914886675897644

cat mse: 0.29058454455327043
xg mse: 0.33871833328434314
gbdt mse: 0.26081918866829057
lr

In [24]:
#LR 融合CatBoostRegressor + LightGBM + xgboost + gbdt现有模型
import pandas as pd
import numpy as np
df = pd.read_csv("happiness_train_complete.csv",encoding="GB2312")
df = df.sample(frac=1,replace=False,random_state=11)
df.reset_index(inplace=True)

df = df[df["happiness"]>0]
Y = df["happiness"]
df["survey_month"] = df["survey_time"].map(lambda line:line.split(" ")[0].split("/")[1]).astype("int64")
df["survey_day"] = df["survey_time"].map(lambda line:line.split(" ")[0].split("/")[2]).astype("int64")
df["survey_hour"] = df["survey_time"].map(lambda line:line.split(" ")[1].split(":")[0]).astype("int64")
X = df.drop(columns=["id","index","happiness","survey_time","edu_other","property_other","invest_other"])

from catboost import Pool, CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.externals import joblib
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.model_selection import train_test_split
kfold = KFold(n_splits=15, shuffle = True, random_state= 12)
catmse = []
lightmse = []
xgmse = []
gbdtmse = []
lrmse = []
i = 0
for train, test in kfold.split(X):
    X_train = X.iloc[train]
    y_train = Y.iloc[train]
    X_test = X.iloc[test]
    y_test = Y.iloc[test]
    
    cat = joblib.load(filename="cat"+str(i))
    light = joblib.load(filename="light"+str(i))
    xg = joblib.load(filename="xg"+str(i))
    gbdt = joblib.load(filename="gbdt"+str(i))
    
    catX = cat.predict(X_test)
    cat_mse = mean_squared_error(y_true=y_test,y_pred=catX)
    print("\ncat mse:",cat_mse)
    catmse.append(cat_mse)

    lightX = light.predict(X_test)
    light_mse = mean_squared_error(y_true=y_test,y_pred=lightX)
    print("light mse:",light_mse)
    lightmse.append(light_mse)
    
    xgX = xg.predict(X_test)
    xg_mse = mean_squared_error(y_true=y_test,y_pred=xgX)
    print("xg mse:",xg_mse)
    xgmse.append(xg_mse)
    
    gbdtX = gbdt.predict(X_test.fillna(-8))
    gbdt_mse = mean_squared_error(y_true=y_test,y_pred=gbdtX)
    print("gbdt mse:",gbdt_mse)
    gbdtmse.append(gbdt_mse)
    
    res = np.c_[catX,lightX,xgX,gbdtX]
    lr = Ridge(fit_intercept=False, alpha=75)
    lr.fit(res,y_test)
    print(lr.coef_)

    y_pred = lr.predict(res)
    lr_mse = mean_squared_error(y_true=y_test,y_pred=y_pred)
    print("lr mse:",lr_mse)
    lrmse.append(lr_mse)
    joblib.dump(filename="lr"+str(i),value=lr)
    i+=1
    
print("\ncatmse:",np.mean(catmse))
print("\n\nlightmse:",np.mean(lightmse))
print("xgmse:",np.mean(xgmse))
print("gbdtmse:",np.mean(gbdtmse))
print("lrmse:",np.mean(lrmse))

    
df2 = pd.read_csv("happiness_test_complete.csv",encoding="GB2312")
df2["survey_month"] = df2["survey_time"].map(lambda line:line.split(" ")[0].split("/")[1]).astype("int64")
df2["survey_day"] = df2["survey_time"].map(lambda line:line.split(" ")[0].split("/")[2]).astype("int64")
df2["survey_hour"] = df2["survey_time"].map(lambda line:line.split(" ")[1].split(":")[0]).astype("int64")
out = df2[["id"]]
X = df2.drop(columns=["id","survey_time","edu_other","property_other","invest_other"])
prediction = []

for i in range(15):
    cat = joblib.load(filename="cat"+str(i))
    light = joblib.load(filename="light"+str(i))
    xg = joblib.load(filename="xg"+str(i))
    gbdt = joblib.load(filename="gbdt"+str(i))
    lr = joblib.load(filename="lr"+str(i))
    
    catX = cat.predict(X)
    lightX = light.predict(X)
    xgX = xg.predict(X)
    gbdtX = gbdt.predict(X.fillna(-8))
    res = np.c_[catX,lightX,xgX,gbdtX]
    prediction.append(lr.predict(res))
    
a = np.array(prediction)
def cut(arr):
    arr2 = []
    for x in arr:
        if x<1:
            arr2.append(1)
        elif x>5:
            arr2.append(5)
        else :
            arr2.append(x)
    return arr2
out["happiness"] = np.mean(np.array(prediction),axis=0)
out.to_csv("happiness_submit.csv",index=False)
print("done")


# out["happiness"] = cut(np.sum((1/np.array(lrmse)*a.T),axis=1)/np.sum(1/np.array(lrmse)))
# out.to_csv("happiness_submit.csv",index=False)
print("done")


cat mse: 0.5140745902446223
light mse: 0.5246388446214709
xg mse: 0.3949933979705159
gbdt mse: 0.5189500888507569
[0.15692732 0.130091   0.56216864 0.1499844 ]
lr mse: 0.441715467691966

cat mse: 0.44963717420374527
light mse: 0.44853289149980635
xg mse: 0.34921170255919454
gbdt mse: 0.4570418891557324
[0.169487   0.17313728 0.51354472 0.14734901]
lr mse: 0.392826862591736

cat mse: 0.40583414150925673
light mse: 0.39798888520938375
xg mse: 0.3192823411725259
gbdt mse: 0.4033749775774016
[0.16202939 0.19026981 0.46212397 0.18036186]
lr mse: 0.3578575641042594

cat mse: 0.46129771045939694
light mse: 0.4653840765647241
xg mse: 0.3557698168444744
gbdt mse: 0.45943222208389095
[0.15867871 0.15014093 0.52070624 0.17347199]
lr mse: 0.399966975754544

cat mse: 0.4155310816111115
light mse: 0.4198606821280977
xg mse: 0.33401968129637316
gbdt mse: 0.4286129037296931
[0.19045141 0.17835247 0.47440746 0.15176293]
lr mse: 0.37433437332477126

cat mse: 0.40941501701181154
light mse: 0.41220334216